In [2]:
import pandas as pd
import sqlite3

# === 1. File paths ===
forest_file = "C:/Users/sidde/Downloads/Bird_Monitoring_Data_FOREST.XLSX"
grassland_file = "C:/Users/sidde/Downloads/Bird_Monitoring_Data_GRASSLAND.XLSX"

# === 2. Load sheet names ===
forest_sheets = pd.ExcelFile(forest_file).sheet_names
grassland_sheets = pd.ExcelFile(grassland_file).sheet_names

# === 3. Load & label each sheet ===
def load_and_label_sheets(file_path, sheet_names, location_type):
    df_list = []
    for sheet in sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet)
        df['Admin_Unit_Code'] = sheet
        df['Location_Type'] = location_type
        df_list.append(df)
    return pd.concat(df_list, ignore_index=True)

forest_data = load_and_label_sheets(forest_file, forest_sheets, "Forest")
grassland_data = load_and_label_sheets(grassland_file, grassland_sheets, "Grassland")

# === 4. Merge datasets ===
full_data = pd.concat([forest_data, grassland_data], ignore_index=True)

# === 5. Clean columns ===
full_data['Date'] = pd.to_datetime(full_data['Date'], errors='coerce')
full_data['Start_Time'] = pd.to_datetime(full_data['Start_Time'], errors='coerce').dt.time
full_data['End_Time'] = pd.to_datetime(full_data['End_Time'], errors='coerce').dt.time
full_data['Temperature'] = pd.to_numeric(full_data['Temperature'], errors='coerce')
full_data['Humidity'] = pd.to_numeric(full_data['Humidity'], errors='coerce')
full_data['Visit'] = pd.to_numeric(full_data['Visit'], errors='coerce')

# Boolean-like columns
bool_columns = [
    'Flyover_Observed', 'PIF_Watchlist_Status', 'Regional_Stewardship_Status',
    'Initial_Three_Min_Cnt'
]
for col in bool_columns:
    full_data[col] = full_data[col].astype(str).str.strip().str.lower().replace({'true': True, 'false': False})
    full_data[col] = full_data[col].astype('boolean')

# === 6. Save to SQLite ===
conn = sqlite3.connect("bird_observations.db")
full_data.to_sql("bird_data", conn, if_exists="replace", index=False)
conn.close()

print("Database 'bird_observations.db' created successfully!")


C:\Users\sidde\AppData\Local\Temp\ipykernel_8760\1334356797.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df_list, ignore_index=True)
C:\Users\sidde\AppData\Local\Temp\ipykernel_8760\1334356797.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_data[col] = full_data[col].astype(str).str.strip().str.lower().replace({'true': True, 'false': False})
C:\Users\sidde\AppData\Local\Temp\ipykernel_8760\1334356797.py:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will 

Database 'bird_observations.db' created successfully!


In [4]:
import sqlite3
import pandas as pd
import numpy as np
import os

# === 1. Setup ===
db_path = "bird_observations.db"
export_folder = "exports"

# Create folder if not exists
os.makedirs(export_folder, exist_ok=True)

# Connect to database
conn = sqlite3.connect(db_path)

# === 2. Queries dictionary (without LOG in Shannon Index) ===
queries = {
    # Habitat & Species Overview
    "total_unique_species": """
        SELECT COUNT(DISTINCT TaxonCode) AS unique_species
        FROM bird_data
        WHERE TaxonCode IS NOT NULL
    """,
    "total_observations": """
        SELECT COUNT(*) AS total_observations
        FROM bird_data
    """,
    "species_count_per_habitat": """
        SELECT Location_Type, COUNT(DISTINCT TaxonCode) AS species_count
        FROM bird_data
        GROUP BY Location_Type
    """,

    # Species Rankings
    "top10_species_overall": """
        SELECT TaxonCode, COUNT(*) AS observations
        FROM bird_data
        WHERE TaxonCode IS NOT NULL
        GROUP BY TaxonCode
        ORDER BY observations DESC
        LIMIT 10
    """,
    "top10_species_per_habitat": """
        SELECT Location_Type, TaxonCode, COUNT(*) AS observations
        FROM bird_data
        WHERE TaxonCode IS NOT NULL
        GROUP BY Location_Type, TaxonCode
        ORDER BY Location_Type, observations DESC
    """,

    # Time & Trend Analysis
    "monthly_observation_trend": """
        SELECT STRFTIME('%m', Date) AS month, COUNT(*) AS observations
        FROM bird_data
        WHERE Date IS NOT NULL
        GROUP BY month
        ORDER BY month
    """,
    "yearly_observation_trend": """
        SELECT STRFTIME('%Y', Date) AS year, COUNT(*) AS observations
        FROM bird_data
        WHERE year IS NOT NULL
        GROUP BY year
        ORDER BY year
    """,
    "seasonal_patterns": """
        SELECT CASE
            WHEN STRFTIME('%m', Date) IN ('12','01','02') THEN 'Winter'
            WHEN STRFTIME('%m', Date) IN ('03','04','05') THEN 'Spring'
            WHEN STRFTIME('%m', Date) IN ('06','07','08') THEN 'Summer'
            WHEN STRFTIME('%m', Date) IN ('09','10','11') THEN 'Autumn'
        END AS season,
        COUNT(*) AS observations
        FROM bird_data
        WHERE Date IS NOT NULL
        GROUP BY season
    """,
    "time_of_day_pattern": """
        SELECT CASE
            WHEN CAST(STRFTIME('%H', Start_Time) AS INTEGER) BETWEEN 5 AND 11 THEN 'Morning'
            WHEN CAST(STRFTIME('%H', Start_Time) AS INTEGER) BETWEEN 12 AND 16 THEN 'Afternoon'
            WHEN CAST(STRFTIME('%H', Start_Time) AS INTEGER) BETWEEN 17 AND 20 THEN 'Evening'
            ELSE 'Night'
        END AS time_of_day,
        COUNT(*) AS observations
        FROM bird_data
        WHERE Start_Time IS NOT NULL
        GROUP BY time_of_day
    """,

    # Threat Status & Conservation
    "threatened_species_per_habitat": """
        SELECT Location_Type, COUNT(DISTINCT TaxonCode) AS threatened_species
        FROM bird_data
        WHERE PIF_Watchlist_Status = 1
        GROUP BY Location_Type
    """,
    "regional_stewardship_species_per_habitat": """
        SELECT Location_Type, COUNT(DISTINCT TaxonCode) AS stewardship_species
        FROM bird_data
        WHERE Regional_Stewardship_Status = 1
        GROUP BY Location_Type
    """,

    # Environmental Factors
    "temperature_vs_species_richness": """
        SELECT ROUND(Temperature, 0) AS temp_rounded, COUNT(DISTINCT TaxonCode) AS species_richness
        FROM bird_data
        WHERE Temperature IS NOT NULL
        GROUP BY temp_rounded
        ORDER BY temp_rounded
    """,
    "humidity_vs_species_richness": """
        SELECT ROUND(Humidity, 0) AS humidity_rounded, COUNT(DISTINCT TaxonCode) AS species_richness
        FROM bird_data
        WHERE Humidity IS NOT NULL
        GROUP BY humidity_rounded
        ORDER BY humidity_rounded
    """,

    # Observer & Location Analysis — Step 1 only (counts)
    "species_diversity_per_admin": """
        SELECT Admin_Unit_Code, TaxonCode, COUNT(*) AS species_count
        FROM bird_data
        WHERE TaxonCode IS NOT NULL
        GROUP BY Admin_Unit_Code, TaxonCode
    """,
    "top_observers": """
        SELECT Observer, COUNT(*) AS total_observations
        FROM bird_data
        WHERE Observer IS NOT NULL
        GROUP BY Observer
        ORDER BY total_observations DESC
        LIMIT 10
    """
}

# === 3. Run all queries except Shannon Index ===
for name, query in queries.items():
    if name != "species_diversity_per_admin":
        df = pd.read_sql_query(query, conn)
        csv_path = os.path.join(export_folder, f"{name}.csv")
        df.to_csv(csv_path, index=False)
        print(f" Exported: {csv_path}")

# === 4. Calculate Shannon Index in Python ===
df_diversity = pd.read_sql_query(queries["species_diversity_per_admin"], conn)
diversity_results = []
for admin_unit, group in df_diversity.groupby("Admin_Unit_Code"):
    total_count = group["species_count"].sum()
    proportions = group["species_count"] / total_count
    shannon_index = -np.sum(proportions * np.log(proportions))
    diversity_results.append({
        "Admin_Unit_Code": admin_unit,
        "species_count": group["species_count"].nunique(),
        "shannon_index": round(shannon_index, 3)
    })

df_shannon = pd.DataFrame(diversity_results)
csv_path = os.path.join(export_folder, "species_diversity_per_admin.csv")
df_shannon.to_csv(csv_path, index=False)
print(f" Exported Shannon Index: {csv_path}")

# === 5. Close connection ===
conn.close()
print("\nAll queries executed and exported successfully!")


 Exported: exports\total_unique_species.csv
 Exported: exports\total_observations.csv
 Exported: exports\species_count_per_habitat.csv
 Exported: exports\top10_species_overall.csv
 Exported: exports\top10_species_per_habitat.csv
 Exported: exports\monthly_observation_trend.csv
 Exported: exports\yearly_observation_trend.csv
 Exported: exports\seasonal_patterns.csv
 Exported: exports\time_of_day_pattern.csv
 Exported: exports\threatened_species_per_habitat.csv
 Exported: exports\regional_stewardship_species_per_habitat.csv
 Exported: exports\temperature_vs_species_richness.csv
 Exported: exports\humidity_vs_species_richness.csv
 Exported: exports\top_observers.csv
 Exported Shannon Index: exports\species_diversity_per_admin.csv

All queries executed and exported successfully!
